In [1]:
using LinearAlgebra

using Qutee
using Qutee.QuantumInfo.KrausDecompose: KrausRepresentation
using Qutee.QuantumInfo.Optimization: kraus2stiefel

# Kraus Operators

By Stinespring Dilation theorem, we may express the evolution of a density matrix by considering a larger space:

$$\mathcal{E}(\rho) = \mathrm{tr}_{env}(\lbrack U(\rho \otimes \rho_{env} )U^{\dagger}\rbrack) = \sum_e K_e \rho K_e^\dagger$$


The following code produces the Kraus Operators given an environment (detector) state $\rho_{env}$ and a unitary matrix.

**Example: Nielsen & Chuang CNOT (8.6, pg. 359)**

Let $U$ be a CNOT gate, and $\rho_{env} = \ket{0}\bra{0}$. Then, the Kraus operators are $P_m = \ket{m}\bra{m}$, giving:
$$\mathcal{E}(\rho) = P_0\rho P_0 + P_1 \rho P_1$$

In [2]:
U = [0 0 1 0; 0 1 0 0; 1 0 0 0; 0 0 0 1]

4×4 Matrix{Int64}:
 0  0  1  0
 0  1  0  0
 1  0  0  0
 0  0  0  1

In [3]:
rho_d = [1 0; 0 0]


2×2 Matrix{Int64}:
 1  0
 0  0

In [4]:
# Obtain a dictionary of Kraus operators, where the key corresponds with $\bra{i}$ and $\ket{e}$

K = KrausRepresentation(U, rho_d)

Dict{Tuple{Int64, Int64}, AbstractMatrix} with 2 entries:
  (1, 1) => [-1.0 0.0; 0.0 0.0]
  (0, 1) => [0.0 0.0; 0.0 -1.0]

In [5]:
# We see that we obtain the correct Kraus operators
K = Array(K)

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 -1.0  0.0
  0.0  0.0

[:, :, 2] =
 0.0   0.0
 0.0  -1.0

In [6]:
# Check if kraus operators are valid
k = K |> kraus2stiefel 
k'k ≈ I(2)

true

In [7]:
# Create a new unitary matrix from the Kraus operators

U_new = zeros(4,4)

U_new[begin:2,begin:2] = K[:,:,2]
U_new[3:end,begin:2] = K[:,:,1]


U_new = Matrix(qr(U_new).Q)

KrausRepresentation(U_new, rho_d) |> Array

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 -1.0  0.0
  0.0  0.0

[:, :, 2] =
 0.0   0.0
 0.0  -1.0

**Example: random $U$ and random $\rho_{env}$**

In [8]:
U = Qutee.QuantumInfo.rand_channel(Array, 1, 4)[:,:,1]

4×4 Matrix{ComplexF64}:
 -0.272216+0.528609im  -0.358824-0.362119im  …   0.188874-0.168065im
  0.164143-0.170149im  -0.350511+0.365915im      0.557268-0.373286im
 -0.417569+0.467054im  -0.205078+0.468629im     -0.363054+0.161154im
  0.325799-0.303197im  -0.438599-0.171244im     -0.522496+0.235392im

In [9]:
rho_d = Qutee.QuantumInfo.Random.rand_density_hs(Array, 2,2)

2×2 Matrix{ComplexF64}:
 0.888466+0.0im       0.163541-0.111438im
 0.163541+0.111438im  0.111534+0.0im

In [10]:
K = KrausRepresentation(U, rho_d)
K = Array(K)

2×2×4 Array{ComplexF64, 3}:
[:, :, 1] =
  -0.135102+0.0450331im  -0.0760302+0.0356121im
 0.00516647-0.133633im    -0.142055+0.121365im

[:, :, 2] =
 -0.138936+0.660309im  0.00683475+0.509343im
  0.165873-0.321796im   -0.549671+0.152368im

[:, :, 3] =
  0.196328+0.545804im  -0.428127-0.129385im
 0.0348905-0.109217im   0.047186+0.385848im

[:, :, 4] =
  0.0624123-0.0461459im  0.0949065-0.00995853im
 -0.0739496-0.119883im    0.101459-0.0516898im

In [11]:
# Check if kraus operators are valid
k = K |> kraus2stiefel 
k'k ≈ I(2)

true